# Highway classifier - Gabbar

## Step 1. Prepare problem

#### 1a. Load libraries

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('ticks')

In [3]:
pd.set_option('display.precision', 2)
pd.set_option('display.max_columns', None)

In [4]:
path = '../downloads/highway-classifier/'

#### 1b. Load dataset

In [5]:
# Loading the training dataset.
training = pd.read_csv(path + 'training.csv')

In [6]:
# Drop all duplicate attributes.
print('Shape before dropping duplicates: {}'.format(training.shape))
training = training.drop_duplicates(subset='changeset_id')
print('Shape after dropping duplicates: {}'.format(training.shape))

Shape before dropping duplicates: (1631, 108)
Shape after dropping duplicates: (1631, 108)


In [7]:
# Creating a smaller sample to speed up workflow.
# training = training[:500]

In [8]:
training.head()

,changeset_id,changeset_harmful,changeset_bbox_area,user_changesets,user_features,user_mapping_days,user_days_since_first_edit,feature_action_create,feature_action_modify,feature_action_delete,feature_version,feature_total_tags,feature_personal_count,feature_similar_tags_count,feature_point,feature_linestring,feature_polygon,feature_distance,feature_line_distance,feature_node_count,feature_node_distance_mean,feature_node_distance_stddev,feature_kinks,feature_area,feature_name_profanity,iD,JOSM,MAPS.ME,Potlatch,Redaction,Vespucci,OsmAnd,Merkaartor,GNOME,aerialway,aeroway,amenity,barrier,boundary,building,craft,emergency,geological,highway,historic,landuse,leisure,man_made,military,natural,office,place,power,public_transport,railway,route,shop,sport,tourism,waterway,residential,service,track,unclassified,footway,path,tertiary,secondary,crossing,primary,bus_stop,turning_circle,other,name,source,surface,tiger:cfcc,tiger:county,tiger:reviewed,oneway,tiger:name_base,maxspeed,lanes,tiger:name_type,ref,service.1,tiger:source,tiger:tlid,tracktype,access,tiger:upload_uuid,yh:WIDTH,tiger:zip_left,tiger:separated,tiger:zip_right,foot,bicycle,yh:TOTYUMONO,yh:WIDTH_RANK,yh:STRUCTURE,yh:TYPE,bridge,layer,lit,crossing.1,tiger:name_direction_prefix,width,other.1
0,47403163,0.0,62.02,9784,828415,1486,3438.0,0,1,0,4,3,0,0,0,1,0,9.87e-03,9.87e-03,2,9.87e-03,0.00,0,38.59,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,8.71,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,47305802,0.0,11209.44,58,573,10,0.0,0,1,0,2,3,0,0,0,1,0,2.51e-01,2.55e-01,4,8.51e-02,0.07,0,19413.46,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,15.06,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,46887225,0.0,0.00,19,63,4,933.0,0,1,0,2,3,0,0,1,0,0,0.00e+00,0.00e+00,1,0.00e+00,0.00,0,0.00,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,1.64,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,47308005,0.0,16205.88,106026,2794751,737,972.0,0,1,0,2,2,0,0,0,1,0,2.65e-01,2.65e-01,2,2.65e-01,0.00,0,21475.87,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.51,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,46472786,0.0,5168.01,11670,375496,1903,2394.0,0,1,0,9,6,0,0,0,1,0,1.55e-01,1.55e-01,2,1.55e-01,0.00,0,8132.71,0,0,0,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,32.55,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


## 2. Summarize data

#### 2a. Descriptive statistics

In [9]:
print('Shape: {}'.format(training.shape))

Shape: (1631, 108)


In [10]:
training.dtypes.head()

changeset_id             int64
changeset_harmful      float64
changeset_bbox_area    float64
user_changesets          int64
user_features            int64
dtype: object

In [11]:
training.describe()

,changeset_id,changeset_harmful,changeset_bbox_area,user_changesets,user_features,user_mapping_days,user_days_since_first_edit,feature_action_create,feature_action_modify,feature_action_delete,feature_version,feature_total_tags,feature_personal_count,feature_similar_tags_count,feature_point,feature_linestring,feature_polygon,feature_distance,feature_line_distance,feature_node_count,feature_node_distance_mean,feature_node_distance_stddev,feature_kinks,feature_area,feature_name_profanity,iD,JOSM,MAPS.ME,Potlatch,Redaction,Vespucci,OsmAnd,Merkaartor,GNOME,aerialway,aeroway,amenity,barrier,boundary,building,craft,emergency,geological,highway,historic,landuse,leisure,man_made,military,natural,office,place,power,public_transport,railway,route,shop,sport,tourism,waterway,residential,service,track,unclassified,footway,path,tertiary,secondary,crossing,primary,bus_stop,turning_circle,other,name,source,surface,tiger:cfcc,tiger:county,tiger:reviewed,oneway,tiger:name_base,maxspeed,lanes,tiger:name_type,ref,service.1,tiger:source,tiger:tlid,tracktype,access,tiger:upload_uuid,yh:WIDTH,tiger:zip_left,tiger:separated,tiger:zip_right,foot,bicycle,yh:TOTYUMONO,yh:WIDTH_RANK,yh:STRUCTURE,yh:TYPE,bridge,layer,lit,crossing.1,tiger:name_direction_prefix,width,other.1
count,1.63e+03,1631.00,1.63e+03,1631.00,1.63e+03,1631.00,1630.00,1.63e+03,1631.00,1631.0,1631.00,1631.00,1.63e+03,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1.63e+03,1.63e+03,1631.00,1631.00,1631.00,1631.00,1631.0,1.63e+03,1631.0,1.63e+03,1631.0,1631.0,1631.0,1.63e+03,1.63e+03,1631.00,1631.0,1631.0,1.63e+03,1631.0,1631.00,1.63e+03,1631.00,1631.00,1631.0,1631.0,1.63e+03,1631.0,1.63e+03,1631.0,1631.00,1.63e+03,1.63e+03,1631.0,1.63e+03,1631.00,1631.0,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1631.00,1.63e+03,1631.0,1631.00,1.63e+03,1631.00,1631.00,1631.00,1631.00,1631.0,1631.00,1631.00,1631.00,1631.0,1631.00,1631.00,1631.00,1631.00,1631.00,1.63e+03,1.63e+03,1.63e+03,1631.00,1.63e+03,1631.0,1631.00,1.63e+03,1631.00,1631.00,1631.00,1631.0,1631.0,1631.0,1631.0,1631.00,1631.00,1631.0,1.63e+03,1631.00,1631.00,1631.00
mean,4.74e+07,0.02,3.69e+06,7642.74,3.14e+05,312.37,1107.75,2.45e-03,1.00,0.0,5.80,5.71,6.74e-03,0.27,0.05,0.93,0.02,0.58,0.81,14.08,0.06,0.03,0.02,6.13e+06,3.07e-03,0.29,0.13,0.01,0.02,0.0,6.74e-03,0.0,6.13e-04,0.0,0.0,0.0,2.15e-04,4.72e-04,0.11,0.0,0.0,4.43e-03,0.0,14.14,1.16e-03,0.02,0.04,0.0,0.0,7.11e-04,0.0,5.52e-05,0.0,0.65,3.18e-03,7.97e-04,0.0,4.29e-04,0.02,0.0,0.35,0.05,0.02,0.07,0.04,0.02,0.08,0.07,3.68e-03,0.1,0.03,1.23e-03,0.16,0.79,0.15,0.60,0.1,0.11,0.06,0.42,0.1,0.33,0.17,0.09,0.15,0.02,1.84e-03,1.84e-03,9.20e-03,0.04,1.23e-03,0.0,0.07,1.23e-03,0.07,0.07,0.09,0.0,0.0,0.0,0.0,0.03,0.04,0.1,4.29e-03,0.02,0.02,2.06
std,4.79e+05,0.14,4.64e+07,24217.77,9.90e+05,521.48,1191.60,4.95e-02,0.05,0.0,4.89,3.23,8.19e-02,0.87,0.21,0.25,0.14,4.53,5.82,65.04,0.40,0.28,0.26,9.47e+07,5.53e-02,0.46,0.34,0.10,0.15,0.0,8.19e-02,0.0,2.48e-02,0.0,0.0,0.0,8.67e-03,1.91e-02,3.13,0.0,0.0,1.79e-01,0.0,13.92,2.10e-02,0.60,0.82,0.0,0.0,2.87e-02,0.0,2.23e-03,0.0,5.67,1.29e-01,3.22e-02,0.0,1.23e-02,0.43,0.0,0.48,0.21,0.15,0.25,0.21,0.14,0.28,0.25,6.06e-02,0.3,0.17,3.50e-02,0.37,0.41,0.36,0.49,0.3,0.31,0.23,0.49,0.3,0.47,0.37,0.29,0.36,0.14,4.29e-02,4.29e-02,9.55e-02,0.20,3.50e-02,0.0,0.26,3.50e-02,0.25,0.26,0.29,0.0,0.0,0.0,0.0,0.17,0.20,0.3,6.54e-02,0.15,0.13,1.60
min,4.48e+07,0.00,0.00e+00,0.00,0.00e+00,0.00,-27.00,0.00e+00,0.00,0.0,2.00,1.00,0.00e+00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00e+00,0.00e+00,0.00,0.00,0.00,0.00,0.0,0.00e+00,0.0,0.00e+00,0.0,0.0,0.0,0.00e+00,0.00e+00,0.00,0.0,0.0,0.00e+00,0.0,0.00,0.00e+00,0.00,0.00,0.0,0.0,0.00e+00,0.0,0.00e+00,0.0,0.00,0.00e+00,0.00e+00,0.0,0.00e+00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00e+00,0.0,0.00,0.00e+00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,0.00,0.00e+00,0.0,0.00,0.00e+00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00

In [12]:
training.groupby('changeset_harmful').size()

changeset_harmful
0.0    1599
1.0      32
dtype: int64

In [13]:
training.corr(method='pearson').head()

,changeset_id,changeset_harmful,changeset_bbox_area,user_changesets,user_features,user_mapping_days,user_days_since_first_edit,feature_action_create,feature_action_modify,feature_action_delete,feature_version,feature_total_tags,feature_personal_count,feature_similar_tags_count,feature_point,feature_linestring,feature_polygon,feature_distance,feature_line_distance,feature_node_count,feature_node_distance_mean,feature_node_distance_stddev,feature_kinks,feature_area,feature_name_profanity,iD,JOSM,MAPS.ME,Potlatch,Redaction,Vespucci,OsmAnd,Merkaartor,GNOME,aerialway,aeroway,amenity,barrier,boundary,building,craft,emergency,geological,highway,historic,landuse,leisure,man_made,military,natural,office,place,power,public_transport,railway,route,shop,sport,tourism,waterway,residential,service,track,unclassified,footway,path,tertiary,secondary,crossing,primary,bus_stop,turning_circle,other,name,source,surface,tiger:cfcc,tiger:county,tiger:reviewed,oneway,tiger:name_base,maxspeed,lanes,tiger:name_type,ref,service.1,tiger:source,tiger:tlid,tracktype,access,tiger:upload_uuid,yh:WIDTH,tiger:zip_left,tiger:separated,tiger:zip_right,foot,bicycle,yh:TOTYUMONO,yh:WIDTH_RANK,yh:STRUCTURE,yh:TYPE,bridge,layer,lit,crossing.1,tiger:name_direction_prefix,width,other.1
changeset_id,1.00,0.02,1.25e-01,-4.15e-02,-5.72e-02,-9.52e-02,-0.08,2.30e-02,-2.30e-02,NaN,-0.03,5.45e-03,-5.43e-03,8.88e-03,-0.08,0.06,0.02,8.43e-02,1.03e-01,8.25e-02,6.29e-02,0.07,5.64e-03,8.82e-02,1.12e-01,-0.14,-1.91e-01,-8.09e-03,-3.24e-02,NaN,-6.02e-02,NaN,7.07e-02,NaN,NaN,NaN,-4.90e-02,1.89e-03,-3.36e-02,NaN,NaN,-3.68e-02,NaN,0.02,1.40e-02,8.18e-02,3.51e-02,NaN,NaN,1.43e-02,NaN,5.26e-02,NaN,-7.26e-02,7.20e-04,-2.31e-02,NaN,5.25e-02,-4.76e-02,NaN,0.03,-0.09,-0.09,6.97e-02,3.12e-02,-8.87e-03,-1.94e-02,3.74e-02,2.48e-02,6.61e-02,-6.34e-02,-2.35e-02,-0.05,0.04,1.18e-02,0.23,-1.09e-02,-2.06e-02,-1.45e-02,0.03,-9.22e-03,4.88e-02,-2.96e-02,6.67e-03,0.03,-0.04,2.87e-02,2.87e-02,7.64e-03,-1.45e-03,1.25e-02,NaN,0.01,2.42e-02,0.01,-2.06e-02,1.07e-02,NaN,NaN,NaN,NaN,-0.02,-0.02,-0.03,4.14e-02,-0.05,-1.43e-03,-7.84e-02
changeset_harmful,0.02,1.00,1.51e-01,-4.03e-02,-1.92e-02,-4.73e-02,-0.10,-7.01e-03,7.01e-03,NaN,0.05,-4.90e-02,4.24e-02,-8.18e-03,0.03,-0.07,0.07,1.16e-01,2.08e-01,1.97e-01,4.53e-03,0.02,2.32e-02,5.93e-02,7.21e-02,0.15,-1.70e-02,6.97e-02,-2.13e-02,NaN,-1.17e-02,NaN,-3.50e-03,NaN,NaN,NaN,-3.50e-03,-3.50e-03,-5.03e-03,NaN,NaN,-3.50e-03,NaN,-0.05,-7.84e-03,2.52e-01,1.72e-01,NaN,NaN,1.75e-01,NaN,-3.50e-03,NaN,2.35e-02,-3.50e-03,-3.50e-03,NaN,-4.96e-03,8.23e-02,NaN,-0.06,0.01,0.04,-2.11e-03,1.19e-01,1.19e-02,-2.65e-02,-3.82e-02,6.44e-02,-1.74e-02,4.87e-04,-4.96e-03,0.03,-0.08,2.64e-03,-0.11,-2.35e-03,-5.66e-03,4.53e-03,-0.06,-3.22e-02,-6.23e-02,-2.75e-02,-3.06e-02,-0.06,0.01,-6.07e-03,-6.07e-03,3.27e-02,-7.77e-03,-4.96e-03,NaN,-0.02,-4.96e-03,-0.02,-3.97e-02,-2.89e-02,NaN,NaN,NaN,NaN,-0.02,-0.03,-0.02,5.83e-02,-0.02,1.53e-02,4.74e-02
changeset_bbox_area,0.13,0.15,1.00e+00,-3.04e-03,1.06e-03,-3.19e-03,-0.02,-3.94e-03,3.94e-03,NaN,0.11,-2.54e-02,-6.53e-03,1.10e-03,-0.02,0.02,-0.01,9.32e-01,9.02e-01,3.85e-01,5.28e-01,0.67,-5.92e-03,8.85e-01,-4.39e-03,0.07,8.71e-03,-8.40e-03,-1.17e-02,NaN,-6.55e-03,NaN,4.61e-01,NaN,NaN,NaN,-1.96e-03,-1.97e-03,-2.79e-03,NaN,NaN,-1.97e-03,NaN,-0.06,-4.41e-03,-2.81e-03,-3.93e-03,NaN,NaN,-1.97e-03,NaN,-1.97e-03,NaN,-9.07e-03,-1.95e-03,-9.45e-04,NaN,-2.21e-03,-3.97e-03,NaN,-0.06,-0.02,0.03,1.39e-02,6.34e-02,-1.05e-02,-2.17e-02,-1.93e-03,-4.84e-03,4.14e-02,-1.42e-02,-2.79e-03,0.02,-0.10,1.10e-01,-0.07,1.09e-02,8.68e-03,2.92e-02,-0.04,1.07e-02,-5.38e-02,-3.44e-02,1.21e-02,0.05,-0.01,-3.41e-03,-3.41e-03,5.54e-02,-1.43e-02,-2.79e-03,NaN,-0.02,-2.79e-03,-0.02,-2.16e-02,-2.38e-02,NaN,NaN,NaN,NaN,-0.01,-0.02,-0.03,-5.22e-03,-0.01,-9.56e-03,1.03e-02
user_changesets,-0.04,-0.04,-3.04e-03,1.00e+00,8.09e-01,4.13e-01,0.12,-1.53e-02,1.53e-02,NaN,-0.05,-8.33e-02,3.11e-03,-4.76e-02,-0.04,0.05,-0.03,9.30e-03,-1.79e-03,-1.76e-02,4.78e-02,0.02,-1.96e-02,-4.67e-03,1.89e-02,-0

In [14]:
skew = training.skew()
skew.sort(ascending=False)
skew.head()

place      40.39
natural    40.39
route      40.39
barrier    40.39
railway    40.39
dtype: float64

#### 2b. Data visualizations

In [15]:
# Histograms.
# training.hist(figsize=(20, 20));

In [16]:
# Density plot.
# training.plot.density(figsize=(20, 50), subplots=True, sharex=False);

In [17]:
# training.plot.box(layout=(5, 4), figsize=(20, 25), subplots=True, sharex=False, sharey=False);

In [18]:
# sns.heatmap(training.corr(method='pearson'));

## 3. Prepare data

#### 3a. Data cleaning

#### 3b. Feature selection

In [19]:
non_training_attributes = ['changeset_id', 'changeset_harmful']
training_attributes = list(set(training.columns) - set(non_training_attributes))

# Drop rows with any null in training attributes.
training = training.dropna(subset=training_attributes)

X = training.drop(non_training_attributes, axis=1)
y = training['changeset_harmful']

#### 3c. Data transforms

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [21]:
# Estimate importance of all features.
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(X_scaled, y)

importances = pd.DataFrame(
    list(zip(X.columns, model.feature_importances_)),
    columns=['feature', 'importance']
)
importances.sort(columns='importance', ascending=False).head()

,feature,importance
16,feature_line_distance,0.07
43,landuse,0.05
0,changeset_bbox_area,0.05
17,feature_node_count,0.05
1,user_changesets,0.05


## 4. Evaluate algorithms

#### 4a. Split-out validation dataset
- We have a seperate validation dataset. Yay!

#### 4b. Spot Check Algorithms
- Running algorithms in a loop below.

#### 4c. Compare Algorithms

In [22]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

models = []
models.append(('LR', LogisticRegression()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('SVM', SVC()))
models.append(('RFC', RandomForestClassifier()))
models.append(('GBC', GradientBoostingClassifier()))

results = []
names = []
for (name, model) in models:
    kfold = KFold(n_splits=2, random_state=7)
    result = cross_val_score(model, X_scaled, y, cv=kfold, scoring='roc_auc')
    results.append(result)
    names.append(name)
    
    print('{}: {} ({})'.format(name, round(result.mean(), 2), round(result.std(), 2)))

LR: 0.79 (0.03)
CART: 0.59 (0.03)
KNN: 0.62 (0.0)
SVM: 0.8 (0.0)
RFC: 0.68 (0.05)
GBC: 0.84 (0.01)


In [23]:
# NOTE: Temporarily turning off the graph.
# fig, ax = plt.subplots(1, 1)
# plt.boxplot(results)
# ax.set_xticklabels(names)
# ax.set_ylabel('roc_auc')
# ax.set_ylim((0, 1));

## 5. Improve Accuracy

#### 5a. Algorithm Tuning

In [24]:
harmful_count = training[training['changeset_harmful'] == True].shape[0]
not_harmful_count = training[training['changeset_harmful'] == False].shape[0]

# Making this a float to prevent truncation due to integer division.
count = 1.0 * (harmful_count + not_harmful_count)

# Calculate sample weights based on number of occourances.
sample_weight = [count if 10 else 1 for l in y]

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
param_grid = {
    'n_estimators': [300, 500, 700],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1,  0.5],
    'max_features': ['log2'],
    'random_state': [5],
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, fit_params={'sample_weight': sample_weight})
grid.fit(X_scaled, y)

print('Best score: {}'.format(grid.best_score_))
print('Best params: {}'.format(grid.best_params_))

Best score: 0.9815950920245399
Best params: {'max_features': 'log2', 'n_estimators': 300, 'random_state': 5, 'learning_rate': 0.05, 'max_depth': 7}


In [26]:
model = grid.best_estimator_
y_model = model.predict(X_scaled)

In [27]:
training['prediction'] = model.predict(X_scaled)
training.to_csv(path + 'training-review.csv', index=False)

#### 5b. Test options and evaluation metric

In [28]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(y, y_model)

matrix = pd.DataFrame(matrix, index=['Labelled good', 'Labelled harmful'], columns=['Predicted good', 'Predicted harmful'])
matrix

,Predicted good,Predicted harmful
Labelled good,1598,0
Labelled harmful,0,32


In [29]:
total = matrix.sum().sum()
matrix / total

,Predicted good,Predicted harmful
Labelled good,0.98,0.00
Labelled harmful,0.00,0.02


In [30]:
from sklearn.metrics import classification_report

report = classification_report(y, y_model)
print(report)

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      1598
        1.0       1.00      1.00      1.00        32

avg / total       1.00      1.00      1.00      1630



In [31]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_scaled, y, cv=kfold, scoring='roc_auc')

# An area of 0.5 represents a model that is as good as random.
# An area of 1.0 represents a model that made all predictions perfectly.
print('Scores: {} ({})'.format(round(scores.mean(), 2), round(scores.std(), 2)))

Scores: 0.84 (0.02)


#### 5c. Ensembles
- Todo

## 6. Finalize model

#### 6a. Predictions on validation dataset

In [32]:
validation = pd.read_csv(path + 'validation.csv')
validation.head()

,changeset_id,changeset_harmful,changeset_bbox_area,user_changesets,user_features,user_mapping_days,user_days_since_first_edit,feature_action_create,feature_action_modify,feature_action_delete,feature_version,feature_total_tags,feature_personal_count,feature_similar_tags_count,feature_point,feature_linestring,feature_polygon,feature_distance,feature_line_distance,feature_node_count,feature_node_distance_mean,feature_node_distance_stddev,feature_kinks,feature_area,feature_name_profanity,iD,JOSM,MAPS.ME,Potlatch,Redaction,Vespucci,OsmAnd,Merkaartor,GNOME,aerialway,aeroway,amenity,barrier,boundary,building,craft,emergency,geological,highway,historic,landuse,leisure,man_made,military,natural,office,place,power,public_transport,railway,route,shop,sport,tourism,waterway,residential,service,track,unclassified,footway,path,tertiary,secondary,crossing,primary,bus_stop,turning_circle,other,name,source,surface,tiger:cfcc,tiger:county,tiger:reviewed,oneway,tiger:name_base,maxspeed,lanes,tiger:name_type,ref,service.1,tiger:source,tiger:tlid,tracktype,access,tiger:upload_uuid,yh:WIDTH,tiger:zip_left,tiger:separated,tiger:zip_right,foot,bicycle,yh:TOTYUMONO,yh:WIDTH_RANK,yh:STRUCTURE,yh:TYPE,bridge,layer,lit,crossing.1,tiger:name_direction_prefix,width,other.1
0,46399560,0.0,5082.23,7,1233,2,321.0,0,1,0,2,7,0,0,0,1,0,0.00,0.81,76,0.01,5.53e-03,0,3.28e+04,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,6.20,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,2
1,47480404,0.0,671.61,17,21,3,-5.0,0,1,0,6,5,0,0,0,1,0,0.04,0.04,4,0.01,7.16e-03,0,4.42e+02,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,1.81,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,47921583,0.0,5377.24,41,103,3,0.0,0,1,0,7,8,0,2,0,1,0,1.18,1.18,26,0.05,3.84e-02,0,8.06e+03,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,4.13,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,46472513,0.0,606143.65,85,1961,9,95.0,0,1,0,6,6,0,0,0,1,0,1.68,3.09,87,0.04,2.34e-02,4,1.79e+06,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,6.20,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3
4,47308236,0.0,13051.06,106026,2794751,737,972.0,0,1,0,2,2,0,0,0,1,0,0.23,0.23,2,0.23,0.00e+00,0,1.73e+04,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.51,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [33]:
# Drop all duplicate attributes.
print('Shape before dropping duplicates: {}'.format(validation.shape))
validation = validation.drop_duplicates(subset='changeset_id')
print('Shape after dropping duplicates: {}'.format(validation.shape))

# Drop rows with any null in training attributes.
validation = validation.dropna(subset=training_attributes)

Shape before dropping duplicates: (700, 108)
Shape after dropping duplicates: (700, 108)


In [34]:
validation.groupby('changeset_harmful').size()

changeset_harmful
0.0    687
1.0     13
dtype: int64

In [35]:
# TODO: Replace with a pipeline
vX = validation.drop(non_training_attributes, axis=1)
vX_scaled = scaler.transform(vX)

vy = validation['changeset_harmful']

In [36]:
vy_model = model.predict(vX_scaled)

In [37]:
validation['prediction'] = model.predict(vX_scaled)
validation.to_csv(path + 'validation-review.csv', index=False)

In [38]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(vy, vy_model)
matrix = pd.DataFrame(matrix, index=['Labelled good', 'Labelled harmful'], columns=['Predicted good', 'Predicted harmful'])
matrix

,Predicted good,Predicted harmful
Labelled good,686,1
Labelled harmful,11,2


In [39]:
from sklearn.metrics import classification_report

report = classification_report(vy, vy_model)
print(report)

             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99       687
        1.0       0.67      0.15      0.25        13

avg / total       0.98      0.98      0.98       700



In [40]:
scores = cross_val_score(model, vX_scaled, vy, cv=kfold, scoring='roc_auc')
print('Scores: {} ({})'.format(round(scores.mean(), 2), round(scores.std(), 2)))

Scores: 0.74 (0.05)


#### 6b. Create standalone model on entire training dataset
- Todo

#### 6c. Predictions on testing dataset

In [41]:
testing = pd.read_csv(path + 'testing.csv')
testing.head()

,changeset_id,changeset_harmful,changeset_bbox_area,user_changesets,user_features,user_mapping_days,user_days_since_first_edit,feature_action_create,feature_action_modify,feature_action_delete,feature_version,feature_total_tags,feature_personal_count,feature_similar_tags_count,feature_point,feature_linestring,feature_polygon,feature_distance,feature_line_distance,feature_node_count,feature_node_distance_mean,feature_node_distance_stddev,feature_kinks,feature_area,feature_name_profanity,iD,JOSM,MAPS.ME,Potlatch,Redaction,Vespucci,OsmAnd,Merkaartor,GNOME,aerialway,aeroway,amenity,barrier,boundary,building,craft,emergency,geological,highway,historic,landuse,leisure,man_made,military,natural,office,place,power,public_transport,railway,route,shop,sport,tourism,waterway,residential,service,track,unclassified,footway,path,tertiary,secondary,crossing,primary,bus_stop,turning_circle,other,name,source,surface,tiger:cfcc,tiger:county,tiger:reviewed,oneway,tiger:name_base,maxspeed,lanes,tiger:name_type,ref,service.1,tiger:source,tiger:tlid,tracktype,access,tiger:upload_uuid,yh:WIDTH,tiger:zip_left,tiger:separated,tiger:zip_right,foot,bicycle,yh:TOTYUMONO,yh:WIDTH_RANK,yh:STRUCTURE,yh:TYPE,bridge,layer,lit,crossing.1,tiger:name_direction_prefix,width,other.1
0,48458577,NaN,1.68e+06,352,15033,55,597.0,0,1,0,7,1,0,0,0,1,0,1.53e+00,2.07e+00,47,4.49e-02,0.05,0,1.18e+06,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,11.37,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,48300306,NaN,1.64e+03,39,60,7,119.0,0,1,0,2,5,0,0,0,1,0,5.10e-02,5.10e-02,2,5.10e-02,0.00,0,1.13e+03,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,15.06,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1
2,48392159,NaN,2.52e-01,3943,155606,73,90.0,0,1,0,2,1,0,0,0,1,0,6.11e-03,6.11e-03,2,6.11e-03,0.00,0,9.53e-01,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,15.06,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,48295980,NaN,4.26e+05,25973,976168,1933,2878.0,0,1,0,4,7,0,2,0,1,0,8.34e-01,2.61e+00,58,4.59e-02,0.03,0,6.16e+05,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,11.37,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,48348606,NaN,1.04e+07,12,198,1,0.0,0,1,0,2,1,0,0,0,1,0,5.34e+00,5.58e+00,34,1.69e-01,0.21,0,1.12e+07,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.28,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [42]:
# Drop all duplicate attributes.
print('Shape before dropping duplicates: {}'.format(testing.shape))
testing = testing.drop_duplicates(subset='changeset_id')
print('Shape after dropping duplicates: {}'.format(testing.shape))

Shape before dropping duplicates: (6700, 108)
Shape after dropping duplicates: (6700, 108)


In [43]:
testing = testing.dropna(subset=training_attributes)
print('After samples rows with null: {}'.format(testing.shape))

After samples rows with null: (6700, 108)


In [44]:
# TODO: Replace with a pipeline
tX = testing.drop(non_training_attributes, axis=1)
tX_scaled = scaler.transform(tX)
# ty = testing['changeset_harmful']

In [45]:
testing['prediction'] = model.predict(tX_scaled)

In [46]:
tharmful_count = testing[testing['prediction'] == True].shape[0]
tnot_harmful_count = testing[testing['prediction'] == False].shape[0]

print('Predicted good: {}'.format(tnot_harmful_count))
print('Predicted harmful: {}'.format(tharmful_count))

print('Percentage harmful: {}%'.format(round(100.0 * tharmful_count / testing.shape[0], 2)))

Predicted good: 6692
Predicted harmful: 8
Percentage harmful: 0.12%


In [47]:
testing.to_csv(path + 'testing-review.csv', index=False)

#### 6d. Save model for later use

In [48]:
from sklearn.externals import joblib

model_path = '../gabbar/trained/model.pkl'
joblib.dump(model, model_path)

['../gabbar/trained/model.pkl']